Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML: https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Beautiful Soup

**Beautiful Soup** es una librería **Python** que permite extraer información de contenido en formato **HTML o XML**. Para usarla, es necesario especificar un **parser**, que es responsable de transformar un documento HTML o XML en un árbol complejo de objetos Python. Esto permite, por ejemplo, que podamos interactuar con los elementos de una página web como si estuviésemos utilizando las herramientas del desarrollador de un navegador.

A la hora de extraer información de una web, uno de los parsers más utilizado es el parser HTML de **lxml**. Precisamente, será el que utilicemos en este tutorial.

**Será necesario instalar las siguientes librerías** (si no las tienes ya):

        pip3 install beautifulsoup4 requests pandas

        pip3 install beautifulsoup4 

        pip3 install requests

        pip3 install pandas 

###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

Algunos ejemplos de Web Scraping utilizando Beautiful Soup:

https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/

http://omz-software.com/pythonista/docs/ios/beautifulsoup_guide.html

https://towardsdatascience.com/top-5-beautiful-soup-functions-7bfe5a693482

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [3]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [3]:
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'lxml')

In [27]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_='product-image')
print(len(cervezas_grid))

12


In [5]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_URLs = []
for cerveza in cervezas_grid:
    # print('Cerveza {} de {}, pag {}/{}'.format(count_beer, len(cervezas_grid), page, len(pages)))
    # Para cada cerveza obtenemos su URL, de donde extraeremos toda la info   
    URL_cerveza = cerveza.find('a')['href']
    lista_URLs.append(URL_cerveza)
lista_URLs   

['https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html',
 'https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/botella-33cl/199-orval.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/165-laugar-epa.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html']

In [6]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_URLs[0])
# Creamos una sopa específica con la info de cada cerveza
soup_cerveza = BeautifulSoup(r.text, 'lxml')

In [7]:
# Nombre
name = soup_cerveza.find(class_='h1 product-detail-name').text
print(name)

SanFrutos Vikingathor Boris Brew


In [8]:
# Precio
price = soup_cerveza.find(class_='current-price').find('span')['content']
print(price)

3.05


In [9]:
# Descripcion corta
try:
    descr_short = soup_cerveza.find(class_='description-short').find('p').text
except:
    descr_short = None
print(descr_short)

India Munich Dunkel


In [10]:
# Descripción larga
descr_full_list = []
descr_full_p = soup_cerveza.find(class_='product-description').find_all('p')
for sentence in descr_full_p:
    descr_full_list.append(sentence.text)
descr_full = "\n".join(descr_full_list)
print(descr_full)

Vikingathor ya ha sido galardonada con anterioridad con las Medallas de Plata cosechadas en el European Beer Star de 2011 y 2012 y laMedalla de Plata en el Asia Beer Cup 2013.
¿Y qué podemos decir de Vikingathor Boris Brew? Que es una cerveza oscura de baja fermentación lupulizada, con 8,2 grados de alcohol, aunque ligera y de trago fácil, como notaréis al catarla. El lúpulo fresco predomina sobre la malta. Su espuma es cremosa y de color ámbar, con buena retención.
Apelando a la nariz, el aroma predominante es la mandarina y la golosina, que se mezcla con la naranja y matices pináceos. La malta torrefacta que aporta el color queda adecuadamente balanceado.
Y ojo: que Vikingathor marida, además de con carnes rojas y comidas muy especiadas, con quesos muy maduros.




In [11]:
# Imagen
image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']
print(image)

https://www.labirratorium.com/14853-large_default/arriaca-boris-brew-vikingathor-.jpg


In [12]:
 # Brand
try:
    brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
except:
    brand = None
print(brand)

Cerveza Sanfrutos


In [13]:
# Código de barras
try:
    barcode = soup_cerveza.find(class_='product-reference').find('span').text
except:
    barcode = None
print(barcode)

831


In [16]:
# Features
features_dicc = {}
features = soup_cerveza.find(class_ = 'data-sheet')
try:
    features_dt = features.find_all('dt')
    features_dd = features.find_all('dd')
    for feature, value in zip(features_dt, features_dd):
        print(feature.text, value.text)
        features_dicc[feature.text] = value.text
except:
    features_dicc = {}

Estilo DUNKEL
Origen Segovia
% Alc. ALTO (6-9%)
% Alc. Exacto 8.2
Volumen (cl) 33 Cl
Tipo Fermentación Lager (Baja Fermentación)
IBU 50-75 Amargor alto
IBU Exacto 75
Color Marrón
Envase Botella


In [17]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
count = 1
id_cerv = 'lbt_' + str(count)

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [18]:
# Agregamos a una lista
lista_cervezas = []
lista_cervezas.append([
    ('id', id_cerv),
    ('name', name), 
    ('price', price),
    ('descr_short', descr_short),
    ('descr_full', descr_full),
    ('image', image),
    ('brand', brand),
    ('barcode', barcode),
    ('features', features_dicc)
    ])       
lista_cervezas

[[('id', 'lbt_1'),
  ('name', 'SanFrutos Vikingathor Boris Brew'),
  ('price', '3.05'),
  ('descr_short', 'India Munich Dunkel'),
  ('descr_full',
   'Vikingathor ya ha sido galardonada con anterioridad con las\xa0Medallas de Plata\xa0cosechadas en el\xa0European Beer Star\xa0de 2011 y 2012 y laMedalla de Plata\xa0en el\xa0Asia Beer Cup\xa02013.\n¿Y qué podemos decir de Vikingathor Boris Brew? Que es una cerveza oscura de baja fermentación lupulizada, con 8,2 grados de alcohol, aunque ligera y de trago fácil, como notaréis al catarla. El lúpulo fresco predomina sobre la malta. Su espuma es cremosa y de color ámbar, con buena retención.\nApelando a la nariz, el aroma predominante es la mandarina y la golosina, que se mezcla con la naranja y matices pináceos. La malta torrefacta que aporta el color queda adecuadamente balanceado.\nY ojo: que Vikingathor marida, además de con carnes rojas y comidas muy especiadas, con quesos muy maduros.\n\n'),
  ('image',
   'https://www.labirratorium.co

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

In [19]:
pages = np.arange(1, 81)
count = 1
lista_cervezas = []

for page in pages:
    
    URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page=' + str(page)
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    cervezas_grid = soup.find_all(class_='product-image')
    
    count_beer = 1 # para el print de seguimiento de descarga
    
    for cerveza in cervezas_grid:
        # Print de seguimiento de descarga:
        print('Cerveza {} de {}, pag {}/{}'.format(
            count_beer, len(cervezas_grid), page, len(pages)))

        URL_cerveza = cerveza.find('a')['href']
        r = requests.get(URL_cerveza)
        soup_cerveza = BeautifulSoup(r.text, 'lxml')
        
        id_cerv = 'lbt_' + str(count)
        
        name = soup_cerveza.find(class_='h1 product-detail-name').text   

        price = soup_cerveza.find(class_='current-price').find('span')['content']

        try:
            descr_short = soup_cerveza.find(class_='description-short').find('p').text
        except:
            descr_short = None

        descr_full_list = []
        descr_full_p = soup_cerveza.find(class_='product-description').find_all('p')
        for sentence in descr_full_p:
            descr_full_list.append(sentence.text)
            
        descr_full = "\n".join(descr_full_list)
        
        image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']

        try:
            brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
        except:
            brand = None

        try:
            barcode = soup_cerveza.find(class_='product-reference').find('span').text
        except:
            barcode = None

        features_dicc = {}
        features = soup_cerveza.find(class_ = 'data-sheet')
        try:
            features_dt = features.find_all('dt')
            features_dd = features.find_all('dd')
            for feature, value in zip(features_dt, features_dd):
                #print(feature.text, value.text)
                features_dicc[feature.text] = value.text
        except:
            features_dicc = {}

        lista_cervezas.append([
            ('id', id_cerv),
            ('name', name), 
            ('price', price),
            ('descr_short', descr_short),
            ('descr_full', descr_full),
            ('image', image),
            ('brand', brand),
            ('barcode', barcode),
            ('features', features_dicc)
        ])        
        
        # Pasamos al siguiente id
        count += 1
        count_beer += 1

Cerveza 1 de 12, pag 1/80
Cerveza 2 de 12, pag 1/80
Cerveza 3 de 12, pag 1/80
Cerveza 4 de 12, pag 1/80
Cerveza 5 de 12, pag 1/80
Cerveza 6 de 12, pag 1/80
Cerveza 7 de 12, pag 1/80
Cerveza 8 de 12, pag 1/80
Cerveza 9 de 12, pag 1/80
Cerveza 10 de 12, pag 1/80
Cerveza 11 de 12, pag 1/80
Cerveza 12 de 12, pag 1/80
Cerveza 1 de 12, pag 2/80
Cerveza 2 de 12, pag 2/80
Cerveza 3 de 12, pag 2/80
Cerveza 4 de 12, pag 2/80
Cerveza 5 de 12, pag 2/80
Cerveza 6 de 12, pag 2/80
Cerveza 7 de 12, pag 2/80
Cerveza 8 de 12, pag 2/80
Cerveza 9 de 12, pag 2/80
Cerveza 10 de 12, pag 2/80
Cerveza 11 de 12, pag 2/80
Cerveza 12 de 12, pag 2/80
Cerveza 1 de 12, pag 3/80
Cerveza 2 de 12, pag 3/80
Cerveza 3 de 12, pag 3/80
Cerveza 4 de 12, pag 3/80
Cerveza 5 de 12, pag 3/80
Cerveza 6 de 12, pag 3/80
Cerveza 7 de 12, pag 3/80
Cerveza 8 de 12, pag 3/80
Cerveza 9 de 12, pag 3/80


KeyboardInterrupt: 

In [25]:
lista_cervezas[0]

[('id', 'lbt_1'),
 ('name', 'SanFrutos Vikingathor Boris Brew'),
 ('price', '3.05'),
 ('descr_short', 'India Munich Dunkel'),
 ('descr_full',
  'Vikingathor ya ha sido galardonada con anterioridad con las\xa0Medallas de Plata\xa0cosechadas en el\xa0European Beer Star\xa0de 2011 y 2012 y laMedalla de Plata\xa0en el\xa0Asia Beer Cup\xa02013.\n¿Y qué podemos decir de Vikingathor Boris Brew? Que es una cerveza oscura de baja fermentación lupulizada, con 8,2 grados de alcohol, aunque ligera y de trago fácil, como notaréis al catarla. El lúpulo fresco predomina sobre la malta. Su espuma es cremosa y de color ámbar, con buena retención.\nApelando a la nariz, el aroma predominante es la mandarina y la golosina, que se mezcla con la naranja y matices pináceos. La malta torrefacta que aporta el color queda adecuadamente balanceado.\nY ojo: que Vikingathor marida, además de con carnes rojas y comidas muy especiadas, con quesos muy maduros.\n\n'),
 ('image',
  'https://www.labirratorium.com/14853-

### FBI: Top ten criminals

#### Queremos guardar las imágenes de cada fugitivo y que el nombre de cada archivo sea el nombre del fugitivo:

In [50]:
fbi_url = 'https://www.fbi.gov/wanted/topten'

# parser lxml
# parser html.parser
r= requests.get(fbi_url)
soup = BeautifulSoup(r.text, "lxml")

fbi = soup.find_all('img')
fbi

[<img src="++theme++fbigov.theme/uswds-2.9.0/img/icon-dot-gov.svg"/>,
 <img src="++theme++fbigov.theme/uswds-2.9.0/img/icon-https.svg"/>,
 <img alt="Federal Bureau of Investigation Logo" src="https://www.fbi.gov/++theme++fbigov.theme/images/fbibannerseal.png" title="Federal Bureau of Investigation"/>,
 <img alt="ALEXIS FLORES" class="" src="https://www.fbi.gov/wanted/topten/alexis-flores/@@images/image/preview"/>,
 <img alt="JOSE RODOLFO VILLARREAL-HERNANDEZ" class="" src="https://www.fbi.gov/wanted/topten/jose-rodolfo-villarreal-hernandez/@@images/image/preview"/>,
 <img alt="EUGENE PALMER" class="" src="https://www.fbi.gov/wanted/topten/eugene-palmer/@@images/image/preview"/>,
 <img alt="RAFAEL CARO-QUINTERO" class="" src="https://www.fbi.gov/wanted/topten/rafael-caro-quintero/@@images/image/preview"/>,
 <img alt="BHADRESHKUMAR CHETANBHAI PATEL" class="" src="https://www.fbi.gov/wanted/topten/bhadreshkumar-chetanbhai-patel/@@images/image/preview"/>,
 <img alt="ROBERT WILLIAM FISHER" 

In [62]:
for x in fbi:
    print(x.get('alt'))

None
None
Federal Bureau of Investigation Logo
ALEXIS FLORES
JOSE RODOLFO VILLARREAL-HERNANDEZ
EUGENE PALMER
RAFAEL CARO-QUINTERO
BHADRESHKUMAR CHETANBHAI PATEL
ROBERT WILLIAM FISHER
ALEJANDRO ROSALES CASTILLO
ARNOLDO JIMENEZ
JASON DEREK BROWN
YASER ABDEL SAID


In [66]:
names = []
for x in fbi:
    names.append(x.get('alt'))
names

[None,
 None,
 'Federal Bureau of Investigation Logo',
 'ALEXIS FLORES',
 'JOSE RODOLFO VILLARREAL-HERNANDEZ',
 'EUGENE PALMER',
 'RAFAEL CARO-QUINTERO',
 'BHADRESHKUMAR CHETANBHAI PATEL',
 'ROBERT WILLIAM FISHER',
 'ALEJANDRO ROSALES CASTILLO',
 'ARNOLDO JIMENEZ',
 'JASON DEREK BROWN',
 'YASER ABDEL SAID']

In [63]:
imgs = []
for x in fbi:
    imgs.append(x.get("src"))
imgs

['++theme++fbigov.theme/uswds-2.9.0/img/icon-dot-gov.svg',
 '++theme++fbigov.theme/uswds-2.9.0/img/icon-https.svg',
 'https://www.fbi.gov/++theme++fbigov.theme/images/fbibannerseal.png',
 'https://www.fbi.gov/wanted/topten/alexis-flores/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/jose-rodolfo-villarreal-hernandez/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/eugene-palmer/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/rafael-caro-quintero/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/bhadreshkumar-chetanbhai-patel/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/robert-william-fisher/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/alejandro-castillo/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/arnoldo-jimenez/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/jason-derek-brown/@@images/image/preview',
 'https://www.fbi.gov/wanted/topten/yaser-abdel-said/@@images/image/preview

In [70]:
dict_fbi = zip(names,imgs)
dict_fbi = dict(dict_fbi)
dict_fbi

{None: '++theme++fbigov.theme/uswds-2.9.0/img/icon-https.svg',
 'Federal Bureau of Investigation Logo': 'https://www.fbi.gov/++theme++fbigov.theme/images/fbibannerseal.png',
 'ALEXIS FLORES': 'https://www.fbi.gov/wanted/topten/alexis-flores/@@images/image/preview',
 'JOSE RODOLFO VILLARREAL-HERNANDEZ': 'https://www.fbi.gov/wanted/topten/jose-rodolfo-villarreal-hernandez/@@images/image/preview',
 'EUGENE PALMER': 'https://www.fbi.gov/wanted/topten/eugene-palmer/@@images/image/preview',
 'RAFAEL CARO-QUINTERO': 'https://www.fbi.gov/wanted/topten/rafael-caro-quintero/@@images/image/preview',
 'BHADRESHKUMAR CHETANBHAI PATEL': 'https://www.fbi.gov/wanted/topten/bhadreshkumar-chetanbhai-patel/@@images/image/preview',
 'ROBERT WILLIAM FISHER': 'https://www.fbi.gov/wanted/topten/robert-william-fisher/@@images/image/preview',
 'ALEJANDRO ROSALES CASTILLO': 'https://www.fbi.gov/wanted/topten/alejandro-castillo/@@images/image/preview',
 'ARNOLDO JIMENEZ': 'https://www.fbi.gov/wanted/topten/arnol

In [73]:
for key, value in dict_fbi.items():
    if not "+" in value:  # or http in img_url
        r = requests.get(value)
        if r.status_code == 200:  # Get OK
            with open(str(key) + ".png", 'wb') as f:
                f.write(r.content)